In [1]:
import sys
sys.path.append("/home/schmid/Text2Head/NPHM")

import torch
from torch.optim import Adam
from utils.pipeline import forward, get_latent_mean_std, get_latent_from_text
from utils.similarity import CLIP_similarity, DINO_similarity
import optuna
import json

torch.cuda.empty_cache()

/home/schmid/miniconda3/envs/nphm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


This notebook serves to find the best hyperparameters for the optimization.
Important:
- to make different runs comparable, we always start from lat_mean
- for the model to be able to yield good results for varying prompts, we use the first 15 samples form the validation set

In [8]:
hparams = {
        'resolution': 120,
        'n_iterations': 120,
        'optimizer_lr': 2e-4,
        'optimizer_weight_decay': 5e-5,
        'lr_scheduler_factor': 0.53,
        'lr_scheduler_patience': 5,
        'lr_scheduler_min_lr': 2.6e-6
    }

dataset_path = '/home/schmid/data/validation_set.json'
with open(dataset_path, 'r') as json_file:
    data = json.load(json_file)

def latent_optimization(hparams):
    all_CLIP_sims = []
    for i in range(15):
        lat_mean, lat_std = get_latent_mean_std()
        prompt = data['heads'][i]['description neutral']
        CLIP_gt = data['heads'][i]['CLIP embedding']
        #DINO_gt = data['heads'][i]['DINO embedding']
        best_latent, _, _, hist = get_latent_from_text(prompt, hparams, init_lat=lat_mean)
        image = hist["images"][-1]
        CLIP_gt_similarity = CLIP_similarity(image, CLIP_gt)
        all_CLIP_sims.append(CLIP_gt_similarity)

    all_CLIP_scores_tensor = torch.stack(all_CLIP_sims)
    print(all_CLIP_scores_tensor)
    # Calculate the average
    average_score = torch.mean(all_CLIP_scores_tensor)

    return average_score

def objective(trial):

    search_space = {
        'resolution': 120,
        'n_iterations': trial.suggest_categorical('n_iterations', [60, 80, 100, 120]),
        'optimizer_lr': trial.suggest_float('optimizer_lr', 1e-4, 1e-3, log=True),
        'optimizer_weight_decay': trial.suggest_float('optimizer_weight_decay', 1e-9, 1e-2, log=True),
        'lr_scheduler_factor': trial.suggest_float('lr_scheduler_factor', 0.5, 0.9),
        'lr_scheduler_patience': trial.suggest_int('lr_scheduler_patience', 3, 10),
        'lr_scheduler_min_lr': trial.suggest_float('lr_scheduler_minlr', 1e-6, 1e-5, log=True),
    }

    hparams.update(search_space)

    return latent_optimization(hparams)

In [9]:
if __name__ == '__main__':
    study = optuna.create_study(storage="sqlite:///../optuna_study_hparams.db", study_name="optim_hparams", direction='maximize', load_if_exists=True)
    study.optimize(objective, n_trials=5)
        
    best_params = study.best_params
    print(best_params)
    # print every hyperparameter and its value in a separate line
    for key, value in best_params.items():
        print(f"{key}: {value}")

[I 2023-12-27 14:15:22,109] Using an existing study with name 'optim_hparams' instead of creating a new one.
  0%|          | 0/80 [00:00<?, ?it/s]

100%|██████████| 80/80 [00:33<00:00,  2.40it/s]
[I 2023-12-27 14:16:28,949] Trial 3 finished with value: 65.3125 and parameters: {'n_iterations': 80, 'optimizer_lr': 0.00011832726126277456, 'optimizer_weight_decay': 3.253248507274845e-06, 'lr_scheduler_factor': 0.5766221832547354, 'lr_scheduler_patience': 10, 'lr_scheduler_minlr': 2.244625304947951e-06}. Best is trial 3 with value: 65.3125.


tensor([[[64.3750]],

        [[66.2500]]], device='cuda:0', dtype=torch.float16)
{'n_iterations': 80, 'optimizer_lr': 0.00011832726126277456, 'optimizer_weight_decay': 3.253248507274845e-06, 'lr_scheduler_factor': 0.5766221832547354, 'lr_scheduler_patience': 10, 'lr_scheduler_minlr': 2.244625304947951e-06}
